In [7]:
! git clone https://github.com/jamesxleon/TensorFlow_fundamentals.git

Cloning into 'TensorFlow_fundamentals'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), 4.15 KiB | 1.04 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [9]:
%cd  TensorFlow_fundamentals

/content/TensorFlow_fundamentals


In this notebook we are going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we are foing to cover:
* Intro to tensors
* Getting info from tensors
* Manipulating tensors
* Tensors & Numpy
* Using @tf.function (to speed up regular Python functions)
* Using GPUs (or TPUs) with tensorFlow
* Exercises to try!

## Intro to tensors

In [2]:
# Import tensorflow
import tensorflow as tf
print(tf.__version__)

2.13.0


In [4]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [5]:
# Check the dimensions of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [6]:
#Create a vector
vector = tf.constant([10,10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [7]:
# Check d of v
vector.ndim

1

In [9]:
# Create a matrix (More than 1D)
matrix = tf.constant([[10,7],
                     [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [10]:
matrix.ndim

2

In [12]:
# Create another matrix
another_matrix = tf.constant([
    [10., 7.0],
    [3., 2.],
    [8., 9.]
], dtype = tf.float16) # specify the data type with dtype parameter. default is 32bit precision (more exact). 16bit takes less space

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [14]:
# The number of dimensions?
another_matrix.ndim #still 2 because the dimensions are the number of values inside the shape

2

In [15]:
# Let's create a tensor
tensor = tf.constant([
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
    [[13, 14, 15], [16, 17, 18]]
])

tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [17]:
# Now we have 3 elements in the shape, so it's logical to have a 3 dimension shape
tensor.ndim

# Matrix and tensors will be used as interchangeable terms throughout the course

3

What we've created so far:
- Scalar: a single number
- Vector: a number with direction (e.g. wind speed and direction)
- Matrix: a 2 dimensional array of numbers
- Tensor: an n-dimensional array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)

### Creating tensors with tf.Variable

In [5]:
# Let's check tf.variable() now
# Create the same tensor with tf.Variable as above which will be a variable tensor
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([10,7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [11]:
# Let's try change one of the elements in out changeable tensor
changeable_tensor[0] #indexing to 0

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [12]:
changeable_tensor[0] = 7 # This not supports item assignment, so this is wrong

TypeError: ignored

In [13]:
# So lets try .assign()
changeable_tensor[0].assign(7)
changeable_tensor # It works now!

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [14]:
# Let's try to change a constant tensor
#unchangeable_tensor_[0] = 7 # It doesn't support item assignment either
#unchangeable_tensor[0].assing() # There is not assign method for constant tensors

> 🔑  This is a demonstration of how we might want to have tensors that change or that don't depending on the context we are working in. A lot of times we won't have to decide on this as TensorFlow most times creates the tensors for our neural networks. However, if in doubt, use tf.constant and change it later if needed

### Creating random tensors

Random tensors are tensors of some arbitrary size which contain random numbers

In [21]:
# Create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2)) # Our tensor should be 2 dimensional

# random.uniform generates numbers from an uniform distribution which is a distribution that has constant probability
# So what is random.normal? outputs random values from a normal distribution
# A normal distribution is a distribution that represents many variables as a symmetrical bell-shaped graph

random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))

#Are they equal?
random_1, random_2, random_1 == random_1

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

In [23]:
### I'll try an uniform random tensor now
random_uniform = tf.random.Generator.from_seed(3)
random_uniform = random_uniform.uniform(shape=(3,2))
random_uniform

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[0.13229859, 0.5348098 ],
       [0.57090175, 0.50970507],
       [0.48252344, 0.15580535]], dtype=float32)>

### What if we want to shuffle the order?

Let's say we are working on an image classification problema and the first 10k images are from a class and the last 5k are from the other. This order might affect what our model learns, lying towards the class with the most appeareances on the first phases of the training while ignoring the last ones. That's why we need to shuffle this values to learn both at the same time.

In [24]:
# Shuffle a tensor (valuable for when you want to shuffle your data so the inherent order doesn't affect learning)
not_shuffled = tf.constant([
    [10,7],
    [3,4],
    [2,5]
    ])

# What is the dimension?
not_shuffled.ndim

2

In [27]:
# Shuffle the not_shuffled tensor
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [10,  7],
       [ 2,  5]], dtype=int32)>

In [28]:
# Let's shuffle again to notice how the order changes every run if there is no seed
tf.random.shuffle(not_shuffled)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [39]:
# Shuffle the not_shuffled tensor using a seed
tf.random.shuffle(not_shuffled, seed=42)
# Still get different results!!!

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 3,  4],
       [ 2,  5],
       [10,  7]], dtype=int32)>

In [40]:
# So how to use a seed?
tf.random.set_seed(42) # Global seed
tf.random.shuffle(not_shuffled, seed=42) # Operation level seed

# Now we get the same result! But why?
# Random seeds rely on two types of seed: global level and operation level seeds

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

 **From the tensorflow docs: **
 Operations that rely on a random seed actually derive it from two seeds: the global and operation-level seeds. This sets the global seed.

Its interactions with operation-level seeds is as follows:

- If neither the global seed nor the operation seed is set: A randomly picked seed is used for this op.
- If the global seed is set, but the operation seed is not: The system deterministically picks an operation seed in conjunction with the global seed so that it gets a unique random sequence. Within the same version of tensorflow and user code, this sequence is deterministic. However across different versions, this sequence might change. If the code depends on particular seeds to work, specify both global and operation-level seeds explicitly.
- If the operation seed is set, but the global seed is not set: A default global seed and the specified operation seed are used to determine the random sequence.
- If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence.

In [41]:
tf.random.shuffle(not_shuffled, seed=42)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 2,  5],
       [10,  7],
       [ 3,  4]], dtype=int32)>

Every time you run an experiment within a NN, depending on the random initial state you might get different results so you must want to know the seeds you've been working with in order to make reproducible experiments

### Other ways to create tensors (Using numpy)

In [43]:
# Create a tensor of all ones
tf.ones([10,7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [46]:
# Create a tensor of all zeros
tf.zeros(shape=(3,4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

> The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU ( much faster for numerical computing)

In [48]:
# NumPy into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a numpy array between 1 and 25
numpy_A

# X = tf.constant(some_matix) # capital for matrix or tensor
# y = tf.constant(some_vector) # non-capital for vector

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [50]:
A = tf.constant(numpy_A)
A

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [55]:
# If we want to convert the numpy_A from tensor shape 24 to a 3D tensor
# The new dimensions must add up to the original elements (i.e. 2*3*4 = 24)
A = tf.constant(numpy_A, shape=(2,3,4)) # This is a tensor(>1D). In order, it converts our vector into 2D, 3D and 4D (as for the args)

B = tf.constant(numpy_A) # This is a vector (1D)

A, B

(<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
 array([[[ 1,  2,  3,  4],
         [ 5,  6,  7,  8],
         [ 9, 10, 11, 12]],
 
        [[13, 14, 15, 16],
         [17, 18, 19, 20],
         [21, 22, 23, 24]]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following elements:
* Shape :: Number of elements of each dimension of a tensor : tensor.shape
* Rank :: Number of tensor dimensions (scalar is rank 0, vector r=1, matrix r=2, tensor r=n) : tensor.ndim
* Axis or Dimension :: A particular dimension (indexing) tensor[0], tensor[:,1] (this one gets all elements in the 0 dimension and then the 1st axis)
* Size :: total items in the tensor : tf.size(tensor)

In [58]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros(shape=(2,3,4,5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [61]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [62]:
rank_4_tensor.ndim, rank_4_tensor.shape, tf.size(rank_4_tensor)

(4, TensorShape([2, 3, 4, 5]), <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [63]:
2 * 3 * 4 * 5

120

In [67]:
# Get varios attributes of our tensor
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0]) # axis are count from the shape list
print("Elements along the last axis: ", rank_4_tensor.shape[-1]) # minus one is useful to get the last index (or counting backwards)
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy()) # A single element for the output with numpy

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [69]:
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [75]:
some_list = [1,2,3,4]
some_list[:2]

[1, 2]

In [74]:
# Get the first 2 elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [81]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :] # the ':' without a number = get the whole thing

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [82]:
rank_4_tensor.shape

TensorShape([2, 3, 4, 5])

In [83]:
rank_4_tensor[:1, :1, :, :1]

<tf.Tensor: shape=(1, 1, 4, 1), dtype=float32, numpy=
array([[[[0.],
         [0.],
         [0.],
         [0.]]]], dtype=float32)>

In [85]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10,7],
                             [3,4]]) # Remember, when in doubt use tf.constant
rank_2_tensor.ndim, rank_2_tensor.shape

(2, TensorShape([2, 2]))

In [88]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [90]:
# Get the last item of each row of our rank_2_tensor
rank_2_tensor[-1:,-1:], rank_2_tensor[-1,-1] # None of this worked, the first is getting the last element from the last row, second one idk

(<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[4]], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=4>)

In [91]:
rank_2_tensor[:, -1] # We want both rows (so all from 1D, ':') and the last from 2D (-1)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [94]:
# To add an extra dimension to our rank_2_tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # ... means every axis as before and add a new axis. Alternative to [:, :, tf.newaxis]
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [95]:
# To add an extra dimension to our rank_2_tensor
rank_4_tensor = rank_3_tensor[..., tf.newaxis] # ... means keep the same as before but make it 3D (3 dots)
rank_4_tensor

<tf.Tensor: shape=(2, 2, 1, 1), dtype=int32, numpy=
array([[[[10]],

        [[ 7]]],


       [[[ 3]],

        [[ 4]]]], dtype=int32)>

In [96]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = -1) #"-1" means expand the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [97]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = 0) # "0" means expand the axis at 0 (1st D)

<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[10,  7],
        [ 3,  4]]], dtype=int32)>

In [98]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = 1) #"1" means expand the axis at 1 (2D (inside 1D))

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]], dtype=int32)>

In [99]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis = 2) #"2" means expand the axis at 2 (3D inside 2D) in this case equals to the final axis

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [100]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

# Manipulating tensors (tensor operations)

Most of times we get info from operating trough tensors

**Basic operations**

`+, -, *, /`

In [107]:
# You can add values to a tensor using the addition operator
tensor = tf.constant(
    [
        [10, 7],
        [3, 4]
    ]
)
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [108]:
# the original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [109]:
# Multiplication also works
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [110]:
# Substraction
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [111]:
# However, if you want to speed up your operations you might want to use the tf.math built in operations. for example:
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [112]:
# Even after using the built in function the original tensor is still unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [119]:
tf.multiply(tensor, tensor) # Pairwise, M1_ij * M2_ij instead of a dot product

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [115]:
# Let's try to see some real difference in runtimes
random_tensor_1 = tf.random.Generator.from_seed(3)
random_tensor_2 = tf.random.Generator.from_seed(7)

random_tensor_1 = tf.random.normal(shape=(100,200,400))
random_tensor_2 = tf.random.normal(shape=(100,200,400))

random_tensor_1, random_tensor_2

(<tf.Tensor: shape=(100, 200, 400), dtype=float32, numpy=
 array([[[-5.59097350e-01, -5.34721375e-01,  2.37303329e+00, ...,
           1.01067054e+00, -7.53074408e-01, -9.76758182e-01],
         [-1.42966795e+00,  1.76292405e-01, -3.92474729e-04, ...,
          -3.79056394e-01, -1.36322117e+00,  5.01162887e-01],
         [-8.65153521e-02,  1.33234775e+00,  3.87389004e-01, ...,
          -1.59642637e+00,  2.46941257e+00,  1.27639794e+00],
         ...,
         [ 7.27385104e-01, -4.30612296e-01, -1.79065585e+00, ...,
           3.76371711e-01,  2.16120914e-01, -1.96867302e-01],
         [-1.39758992e+00,  4.31511551e-01,  1.14158070e+00, ...,
           9.00351048e-01,  1.39498249e-01, -6.00560665e-01],
         [ 1.66722929e+00, -1.03196949e-01,  2.59727925e-01, ...,
          -6.44158542e-01, -1.16303913e-01,  5.78252435e-01]],
 
        [[-2.48680025e-01, -4.31491613e-01, -1.92662585e+00, ...,
          -4.28884119e-01, -6.62429273e-01, -1.64513457e+00],
         [-2.02840304e+00, -1

In [116]:
# First multiply with python operators
random_tensor_1 * random_tensor_2

<tf.Tensor: shape=(100, 200, 400), dtype=float32, numpy=
array([[[-3.67040008e-01,  2.20867574e-01,  8.06772113e-01, ...,
         -1.07057512e+00,  6.60901487e-01,  8.04134488e-01],
        [ 1.36507618e+00,  2.82662392e-01,  5.94451034e-04, ...,
         -1.62071973e-01, -7.80871034e-01, -3.65820110e-01],
        [-5.62091023e-02,  9.66490448e-01, -3.27239484e-01, ...,
          2.68775243e-02,  5.17131150e-01, -1.63533652e+00],
        ...,
        [-1.34097266e+00, -6.94337208e-03,  9.28337276e-01, ...,
         -4.40602899e-01,  4.57870305e-01,  1.38161004e-01],
        [-7.33488679e-01,  8.55326712e-01, -8.07414114e-01, ...,
         -1.66622356e-01,  2.43197873e-01, -3.35510612e-01],
        [ 1.63056564e+00, -9.04683694e-02, -2.48759077e-03, ...,
         -4.83196437e-01, -1.54812008e-01, -4.84593391e-01]],

       [[ 1.68996245e-01,  2.65479922e-01,  6.17370427e-01, ...,
         -5.97267091e-01, -4.76649582e-01, -1.05058682e+00],
        [-1.15300052e-01, -7.89234459e-01, -6.

In [118]:
tf.multiply(random_tensor_1, random_tensor_2)

<tf.Tensor: shape=(100, 200, 400), dtype=float32, numpy=
array([[[-3.67040008e-01,  2.20867574e-01,  8.06772113e-01, ...,
         -1.07057512e+00,  6.60901487e-01,  8.04134488e-01],
        [ 1.36507618e+00,  2.82662392e-01,  5.94451034e-04, ...,
         -1.62071973e-01, -7.80871034e-01, -3.65820110e-01],
        [-5.62091023e-02,  9.66490448e-01, -3.27239484e-01, ...,
          2.68775243e-02,  5.17131150e-01, -1.63533652e+00],
        ...,
        [-1.34097266e+00, -6.94337208e-03,  9.28337276e-01, ...,
         -4.40602899e-01,  4.57870305e-01,  1.38161004e-01],
        [-7.33488679e-01,  8.55326712e-01, -8.07414114e-01, ...,
         -1.66622356e-01,  2.43197873e-01, -3.35510612e-01],
        [ 1.63056564e+00, -9.04683694e-02, -2.48759077e-03, ...,
         -4.83196437e-01, -1.54812008e-01, -4.84593391e-01]],

       [[ 1.68996245e-01,  2.65479922e-01,  6.17370427e-01, ...,
         -5.97267091e-01, -4.76649582e-01, -1.05058682e+00],
        [-1.15300052e-01, -7.89234459e-01, -6.

Damn, still no changes in runtime. But, trust me, it helps. Please not those examples above are element wise multiplications instead of dot products

**Now let's take a look at the concept of matrix multiplications**

One of the most common operations in ML, different from multiplying matrices to a scalar (a simple one for those with a pinch of knowledge in linear algebra), but instead doing dot product multiplications

**There are two rules to matrix multiplication:**

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [122]:
# Matrix multiplication in tensorflow with tf.linalg.matmul
print(tensor)
tf.matmul(tensor, tensor), tf.multiply(tensor, tensor) # Just to have an example of the difference between dot product and element wise multiplication

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


(<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[121,  98],
        [ 42,  37]], dtype=int32)>,
 <tf.Tensor: shape=(2, 2), dtype=int32, numpy=
 array([[100,  49],
        [  9,  16]], dtype=int32)>)

In [123]:
# Python has its own operator for matrix multiplication '@'
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [126]:
# Multiplying tensors of different shapes
# Create a (3, 2) tensor
X = tf.constant([
    [1, 2],
    [3, 4],
    [6, 7]
])

# Create another (3, 4) tensor
Y = tf.constant([
    [7, 8],
    [9, 10],
    [11, 12]
])

X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [6, 7]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [127]:
# Try to matrix multiply
X * Y, tf.matmul(X,Y) # Both will return a size incompatible error.

InvalidArgumentError: ignored

In [130]:
# So how do I make this inner dimensions match?
X.shape, Y.shape, X, Y

(TensorShape([3, 2]),
 TensorShape([3, 2]),
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [6, 7]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

#### If I take the second matrix as my utility to fix this I might want to add a dimension to it. However, this are constant tensors!!! Now what?

 For illustration, here, inside numbers for X 3x2 and Y 3x2 Matrices will be 2 & 3 (3x'2 3'x2) While numbers on the outside is the final matrix shape (3x2).

 However, as numbers on the inside does not match, this matmul cannot be solved

So let's reshape the second matrix again, apparently, even for a constant tensor a reshape() is possible (I guess it is because the content itself does not change)

In [135]:
Y

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 7,  8],
       [ 9, 10],
       [11, 12]], dtype=int32)>

In [133]:
tf.reshape(Y, shape=(2,3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [136]:
# Try the multiplicaition now
X @ tf.reshape(Y, shape=(2,3)) # It works!

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [112, 125, 138]], dtype=int32)>

In [138]:
# Again, it works using tf.matmul as well
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [112, 125, 138]], dtype=int32)>

In [139]:
# This will work even if we change X instead of Y
tf.matmul(tf.reshape(X, shape=(2,3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [159, 176]], dtype=int32)>

In [140]:
# Can do the same with transpose (bc of our dimensions) but the transpose will change numbers in a different way
# Transpose shifts the axes, while reshape just shuffles values around to fit your shape (when possible)
X, tf.transpose(X), tf.reshape(X, shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [6, 7]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 3, 6],
        [2, 4, 7]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[1, 2, 3],
        [4, 6, 7]], dtype=int32)>)

_Some times, not all of them, we would need to reshape our data_

**The dot product**

Matrix multriplication is also referred as the dot product which can be computed using:
- `@`
- `tf.matmul`

Remember that the inner numbers of the matrices to be dot producted need to match, this can be achieved through a reshape or a transposed getting different results for each.

In [141]:
#Lets calc the dot product for X and Y using tf.tensordot()
tf.tensordot(X, tf.transpose(Y), axes=1) # Needs to receive an axius

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 98, 124, 150]], dtype=int32)>

In [142]:
# Perform matmul with Y transposed
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 98, 124, 150]], dtype=int32)>

In [143]:
# Now let's perform matmul with reshape
tf.matmul(X, tf.reshape(Y, shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [112, 125, 138]], dtype=int32)>

In [147]:
# Check values for illustration
print("Initial Y values: ")
print(Y, "\n")

print("Y reshaped to (2,3): ")
print(tf.reshape(Y, shape=(2,3)), "\n")

print("Transposed values of Y: ")
print(tf.transpose(Y), "\n") # This type of "logging" is useful to investigate silent errors (errors that will still get the code running but give unexpected behaviors)

Initial Y values: 
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2,3): 
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Transposed values of Y: 
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32) 



Generally when performing matrix multiplications on two tensors with unmatching shapes you'd transpose the matrix rather than reshaping it
### Now let's check what if we need to change the datatype of a tensor

In [151]:
# Create a tensor with default float32
B = tf.constant([1.7, 7.4])

C = tf.constant([7,10])

B, C

(<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.7, 7.4], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>)

In [154]:
# Change from float32 to float16 for reduced precision. float32 will have a 32 bit precision while float16 and bfloat16 take half the space
B = tf.cast(B, dtype=tf.float16)

C = tf.cast(C, dtype=tf.bfloat16)

B,C

(<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>,
 <tf.Tensor: shape=(2,), dtype=bfloat16, numpy=array([7, 10], dtype=bfloat16)>)

### Let's Aggregate tensors now

Aggregating tensors = condensing them from multiple values to a smaller amount of values

In [159]:
# Get the absolute values (Maybe not the best form of aggregation to start with, but honestly I don't care. I've been coding all day long)
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [157]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>


Let's go through different forms of aggregation:
- Minimum
- Maximum
- The mean
- The sum

`tf uses the reduce_'something' for aggregation'`

In [169]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0,100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([30, 45, 85, 37, 63, 35, 72, 83, 79, 29, 53,  8,  7,  5, 76, 43, 82,
       48, 17, 65, 48,  0, 14, 51,  2, 47, 69, 86, 39, 17, 35, 67, 84, 67,
       25, 37, 50, 68, 75, 33, 89, 23, 13, 69, 73, 91, 83, 11, 59, 25])>

In [170]:
tf.size(E), E.ndim, E.shape

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, 1, TensorShape([50]))

In [171]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [172]:
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=91>

In [173]:
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=48>

In [174]:
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2412>

#### I just figured out I populated the matrix as a normal population instead of properly going truh each of its rows and columns separately, so just skip ahead to the next section

In [186]:
# We can also compute variance using tf operations
E_variance = 0
for i in E:
  E_variance = np.power(i - tf.reduce_mean(E), 2)
E_variance = E_variance / tf.size(E)
E_variance

<tf.Tensor: shape=(), dtype=float64, numpy=10.58>

In [187]:
# Now the std
np.sqrt(E_variance)

3.2526911934581184

#### Here we do it the right (but lazy) way

tf.float32 is usually the default type for most operations

In [188]:
# Now we can actually do the same importing the tensorflow_probability module
import tensorflow_probability as tfp
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=722>

In [200]:
# Now the std (needs dtype set to float (pick the bit precision of your preference, just go ahead))
tf.math.reduce_std(tf.cast(E, dtype=tf.float32)) #btw, reduce_std needs to be called from tf.math.reduce_std, don't ask me why.
# Also happens with tf.math.reduce_variance, so tensorflow_probability might not be truly necessary

<tf.Tensor: shape=(), dtype=float32, numpy=26.871962>

### Let's work the positional maximum and minimum.

**It is gonna be necesarry when the representation probability outputs store values for a classifier in a whole column and we want to find the right index to better represent our class**

- The positional maximum of a tensor is the element-wise maximum of the tensor along a given axis or dimension. It returns a new tensor with the same shape as the original tensor, except for the axis or dimension that is reduced to a single value.

The positional maximum can be computed using the tf.math.maximum function in TensorFlow1.

- The positional minimum of a tensor is the element-wise minimum of the tensor along a given axis or dimension. It returns a new tensor with the same shape as the original tensor, except for the axis or dimension that is reduced to a single value.

In [203]:
# Create a new random tensor for finding pos max and pos min
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [204]:
# Maximum element position
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [205]:
# Index on largest value position
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [207]:
tf.reduce_max(F) # This match (remember we are working on a vector)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [208]:
# Check for equality
assert F[tf.argmax(F)] == tf.reduce_max(F) # Shouldn't return an error if this match

In [209]:
# Minimum element position
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [210]:
# Index pos min
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

In [211]:
# Check
assert F[tf.argmin(F)] == tf.reduce_min(F)

### Can we squeeze our tensor? (removing all single dimensions)

In [213]:
# Create a tensor to get started
tf.random.set_seed(42) # Almost forgot to set the seed
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50)) # Adding some single dimensions right from the start
G # Count the brackets that wrap the 50 dimensions

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [214]:
G.shape # Before squeeze

TensorShape([1, 1, 1, 1, 50])

In [215]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>,
 TensorShape([50]))

### One hot encoding

Integer representation for cualitative values. It is a form of numerical encoding for words to pass our NN.

In [216]:
# Create a list of indices
some_list = [0, 1, 2, 3] # Let's say each index is red, green, blue and purple

# One hot encode indices
tf.one_hot(some_list, depth=4) # depth is supposed to be the number of elements for the encoding

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [218]:
# You rarely do this in practice but you can specify custom values for hot encoding
tf.one_hot(some_list, depth=4, on_value='yo I love deep learning', off_value='I also like to dance') # just for fun, NN love numbers, so please go for 1s and 0s (and everything in between)

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'yo I love deep learning', b'I also like to dance',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'yo I love deep learning',
        b'I also like to dance', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'yo I love deep learning', b'I also like to dance'],
       [b'I also like to dance', b'I also like to dance',
        b'I also like to dance', b'yo I love deep learning']],
      dtype=object)>

### A few more math functions (element wise)
- squaring
- log
- square root

In [220]:
# Create new tensor
H = tf.range(1,10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [221]:
# Get the square
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [225]:
# Get the square root
tf.sqrt(tf.cast(H, dtype=tf.float32)) # This will error if using an int type

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [226]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float32)) # This one also need tf.math, again, idk

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy

Tensorflow interacts beutifully with NumPy arrays. **One of the main differences is that tensors run on GPU or TPU for faster numerical processing**

In [228]:
# Create a tensor from np array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [229]:
# Convert it back to np array
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [230]:
# From tensor to np array
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [232]:
J = tf.constant([3.])
J.numpy()[0] # When we need specific behaviors that either type won't allow we can change the type and do it

3.0

In [233]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])

# Check the types
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### How to find access to GPUS

I'll be working this notebook on colab so, if you are not, just get familiar with it and ask for a GPU under the runtime type tab and changing the hardware accelerator.

Colabs' free version include Nvidia K80s, T4s, P4s and P100s GPUs. Last one being the fastest but you'll be assigned based on availability. If higher power is needed, A100s are available for Colab Pro users and TPUs work in a similar way. When you ask for access to this hardware your runtime will restart.

In [236]:
# this is before asking for a GPU, so all we have is CPU. Only ask for GPU when neede to save computing units
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

> If you have access to a CUDA-enabled GPU, TensorFlow will automatically use it

**Good news is that by now you should have a great understanding over tensorflow fundamentals**

I have some pending exercises that will be soon added to the repo